In [1]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import json
import re
from time import sleep

from tqdm import tqdm
from tqdm.notebook import tqdm

import random 
from datetime import datetime
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

path_sales = r'O:\Отдел стратегического развития\Данные\Avito\suburb\Недвижимость\черновики\Sales.xlsx'
path_supply = r'O:\Отдел стратегического развития\Данные\Avito\suburb\Недвижимость\черновики\Supply.xlsx'

In [53]:
sup = pd.read_excel(path_supply)
sal = pd.read_excel(path_sales)

In [19]:
sal['square_home'] = sal['name'].apply(lambda x: '' if not re.findall(r'([\d|,]+)', x) else re.findall(r'([\d|,]+)', x)[0] )

In [20]:
sal['square'] = sal['name'].apply(lambda x: '' if not re.findall(r'([\d|,]+)', x) else re.findall(r'([\d|,]+)', x)[-1] )

In [21]:
sal[['name', 'square', 'square_home']]

,name,square,square_home
0,Коттедж 290 м² на участке 6 сот.,6,290
1,Коттедж 312 м² на участке 12 сот.,12,312
2,Коттедж 290 м² на участке 12 сот.,12,290
3,Коттедж 290 м² на участке 6 сот.,6,290
4,Дом 210 м² на участке 12 сот.,12,210
...,...,...,...
4186,Дача 42 м² на участке 6 сот.,6,42
4187,"Дом 132,7 м² на участке 16 сот.",16,"132,7"
4188,Дом 208 м² на участке 1 сот.,1,208
4189,"Дом 60 м² на участке 6,9 сот.","6,9",60


In [22]:
# sal.to_excel(path_sales, index = False,
           # engine='xlsxwriter', engine_kwargs={'options' : { 'strings_to_urls': False}} )   

In [39]:
pathp = 'O:\Отдел стратегического развития\Данные\Avito\suburb\Недвижимость'
p = pd.read_excel(fr'{pathp}\pers_info.xlsx')
p1 = pd.read_excel(fr'{pathp}\pers_info1.xlsx')
p2 = pd.read_excel(fr'{pathp}\pers_info2.xlsx')
p3 = pd.read_excel(fr'{pathp}\pers_info3.xlsx')

In [95]:
pall = pd.concat([p3, p2, p1, p])[['url', 'Категория земель', 'Этажей в доме', 'Материал стен']].drop_duplicates('url', keep = 'first')\
.reset_index(drop = True)


In [100]:
pall.to_excel(fr'{pathp}\add_info.xlsx', index = False)

In [99]:
nsup = pd.merge(sal.drop_duplicates(subset= ['url'], keep = 'first'), pall, how = 'left', on = 'url').drop_duplicates(subset = ['url'])
# nsup.to_excel(path_sales, index = False)

In [97]:
sal.drop_duplicates(subset= ['url'], keep = 'first')

,url,id,name,type,seller,square,price,price_sot,timestamp_published,address,...,distance_to_metro,date_published,description,location,latitude,longitude,type_deal,date_supple,sale_date,square_home
0,https://www.avito.ru/sankt-peterburg/doma_dach...,3546881644,Коттедж 290 м² на участке 6 сот.,"Дома, дачи, коттеджи",ANSever,6,25900000,89310,1716531382000,Санкт-Петербург,...,NaN,24 мая 09:16,Продается солнечный и просторный коттедж на уч...,Санкт-Петербург,60.147132,30.026134,sale,24.05.2024,25.6.2024,290
1,https://www.avito.ru/sankt-peterburg/doma_dach...,3547006251,Коттедж 312 м² на участке 12 сот.,"Дома, дачи, коттеджи",ANSever,12,35900000,115064,1716531373000,Санкт-Петербург,...,NaN,24 мая 09:16,Продается солнечный и просторный коттедж на уч...,Санкт-Петербург,60.147132,30.026134,sale,24.05.2024,25.6.2024,312
2,https://www.avito.ru/sankt-peterburg/doma_dach...,3547160387,Коттедж 290 м² на участке 12 сот.,"Дома, дачи, коттеджи",ANSever,12,29800000,102759,1716531389000,Санкт-Петербург,...,NaN,24 мая 09:16,Продается солнечный и просторный коттедж на уч...,Санкт-Петербург,60.147132,30.026134,sale,24.05.2024,25.6.2024,290
3,https://www.avito.ru/sankt-peterburg/doma_dach...,3547449579,Коттедж 290 м² на участке 6 сот.,"Дома, дачи, коттеджи",ANSever,6,27500000,94828,1716531398000,Санкт-Петербург,...,NaN,24 мая 09:16,Продается солнечный и просторный коттедж на уч...,Санкт-Петербург,60.147132,30.026134,sale,24.05.2024,25.6.2024,290
4,https://www.avito.ru/sankt-peterburg/doma_dach...,3995023706,Дом 210 м² на участке 12 сот.,"Дома, дачи, коттеджи",Мир квартир COUNTRY,12,39990000,190429,1718525464000,Санкт-Петербург,...,NaN,16 июня 11:11,Арт. 54637374 Предлагается к продаже Новый Одн...,Санкт-Петербург,60.147880,30.026152,sale,16.06.2024,25.6.2024,210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286,https://www.avito.ru/sankt-peterburg_lomonosov...,3879948939,Дом 300 м² на участке 15 сот.,"Дома, дачи, коттеджи",ТЕРРАСА Загородная Недвижимость,15,23500000,78333,1718509835000,NaN,...,NaN,16 июня 06:50,Id:23649.\nПроект 5К Стандарт.\nЛьготная Ипоте...,Ломоносов,59.894852,29.693794,sale,16.06.2024,25.6.2024,300
2304,https://www.avito.ru/sankt-peterburg_lomonosov...,3577585161,"Коттедж 171,3 м² на участке 9 сот.","Дома, дачи, коттеджи",Татьяна,9,17700000,103327,1716441654000,NaN,...,NaN,23 мая 08:20,Предлагаем вашему вниманию уютный дом для круг...,Ломоносов,59.905644,29.651873,sale,23.05.2024,25.6.2024,"171,3"
2308,https://www.avito.ru/sankt-peterburg_lomonosov...,4235625755,Дом 120 м² на участке 6 сот.,"Дома, дачи, коттеджи",Агентство Недвижимости BR Group,6,9500000,79167,1716491438000,NaN,...,NaN,23 мая 22:10,Арт. 66375836 В продаже Дом просто создан для ...,Ломоносов,59.916830,29.642042,sale,23.05.2024,25.6.2024,120
2309,https://www.avito.ru/sankt-peterburg_lomonosov...,3997305866,Коттедж 225 м² на участке 11 сот.,"Дома, дачи, коттеджи",JCat,11,19800000,88000,1716653047000,NaN,...,NaN,25 мая 19:04,"Продаю коттедж 225 м2, 2024 г. Постройки в к/п...",Ломоносов,59.899658,29.639886,sale,25.05.2024,25.6.2024,225


In [88]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium import webdriver

chrome_options = Options()
options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
chrome_options.add_argument("--log-level=3")
chrome_options.add_argument("--headless")
# --log-level=3 --headless

session = webdriver.Chrome()


In [86]:

def get_pers_info(url):
    session.get(url)
    sleep(random.randint(1, 5))
    # session.refresh()
    # sleep(random.randint(1, 3))
    
    soup = BeautifulSoup(session.page_source, 'lxml')
    values = [a.text for a in soup.select("li[class^='params-paramsList__item']")]

    dct = {
        a.split(': ')[0]:a.split(': ')[1]
            for a in values
        }
    return dct

In [54]:
dt = pd.concat([sup, sal])

In [55]:
seek = dt[dt['Категория земель'].isna()].reset_index(drop = True)

In [60]:
urls = seek['url'].tolist()[::-1]

In [84]:
pers_info = [{'url' : a} for a in urls ]
pers_info[:3]

[{'url': 'https://www.avito.ru/sankt-peterburg/doma_dachi_kottedzhi/dom_76_m_na_uchastke_6_sot._4048994286'},
 {'url': 'https://www.avito.ru/sankt-peterburg_zelenogorsk/doma_dachi_kottedzhi/dom_295_m_na_uchastke_12_sot._2433619127'},
 {'url': 'https://www.avito.ru/sankt-peterburg_zelenogorsk/doma_dachi_kottedzhi/dom_156_m_na_uchastke_10_sot._3449213672'}]

In [90]:
g = get_pers_info(urls[0])

In [98]:
for _ in range(5):
    for a in tqdm(range(len(pers_info))):
        if 'Категория земель' not in pers_info[a]:
            data = get_pers_info(pers_info[a]['url'])
            pers_info[a].update(data)
            # data['url'] = pers_info[a]['url']
            # pers_info += [data]

  0%|          | 0/11820 [00:00<?, ?it/s]

  0%|          | 0/11820 [00:00<?, ?it/s]

  0%|          | 0/11820 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [101]:
len([a for a in pers_info if  'Категория земель' in a])

5605

In [103]:
new_d = pd.DataFrame(pers_info).drop_duplicates('url')
new_d.to_excel('O:\Отдел стратегического развития\Данные\Avito\suburb\Недвижимость\pers_info5.xlsx', index = False)

In [ ]:
for a in tqdm(range(len(new_d['url']))):
    if not new_pers_info[a]:
        data = get_pers_info(dt['url'][a])
        data['url'] = dt['url'][a]
        new_pers_info += [data]

In [103]:
dt.drop_duplicates()

,Unnamed: 0,url,id,name,type,seller,square,price,price_sot,timestamp_published,...,highway,time_to_metro,distance_to_metro,date_published,description,location,latitude,longitude,type_deal,date_supple
0,0,https://www.avito.ru/sankt-peterburg/doma_dach...,3480989002,"Дом 126 м² на участке 4,5 сот.","Дома, дачи, коттеджи",Домакоттеджи,"4,5сот",12600000,100000,1716437193000,...,"Александровское шоссе, 12 км",NaN,NaN,23 мая 07:06,Успейте до конца лета построить свой современн...,Санкт-Петербург,60.145475,30.018243,sale,23.05.2024
1,1,https://www.avito.ru/sankt-peterburg/doma_dach...,2489577969,Дом 150 м² на участке 6 сот.,"Дома, дачи, коттеджи",Домакоттеджи,6сот,9900000,66000,1716437259000,...,"Александровское шоссе, 12 км",NaN,NaN,23 мая 07:07,Продажа напрямую от собственника и застройщика...,Санкт-Петербург,60.146272,30.013872,sale,23.05.2024
2,2,https://www.avito.ru/sankt-peterburg/doma_dach...,3991410367,Дом 130 м² на участке 12 сот.,"Дома, дачи, коттеджи",Мария,12сот,33000000,253846,1716986344000,...,"Александровское шоссе, 12 км",NaN,NaN,29 мая 15:39,"Продается дом, в доме есть все необходимое для...",Санкт-Петербург,60.145730,30.022272,sale,29.05.2024
3,3,https://www.avito.ru/sankt-peterburg/doma_dach...,3320034430,Дом 202 м² на участке 10 сот.,"Дома, дачи, коттеджи",Станислава,10сот,19900000,98515,1718350989000,...,"Александровское шоссе, 12 км",NaN,NaN,14 июня 10:43,"От собственника -Уютный, обжитой дачный посело...",Санкт-Петербург,60.147338,30.009866,sale,14.06.2024
4,4,https://www.avito.ru/sankt-peterburg/doma_dach...,4178517724,Дом 83 м² на участке 12 сот.,"Дома, дачи, коттеджи",АН ЮНОНА,12сот,13970000,168313,1716375082000,...,"Александровское шоссе, 12 км",NaN,NaN,22 мая 13:51,Продается отличный дом в Курортном районе Санк...,Санкт-Петербург,60.149949,29.994918,sale,22.05.2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22973,22973,https://www.avito.ru/sankt-peterburg/doma_dach...,4183887144,Дом 180 м² на участке 20 сот.,"Дома, дачи, коттеджи",Жанна,20сот,23900000,132778,1717586315000,...,NaN,NaN,NaN,5 июня 14:18,"Лен. Обл. Выборгский район, пос. Семиозерье, С...",Санкт-Петербург,60.310266,29.316226,sale,05.06.2024
22974,22974,https://www.avito.ru/sankt-peterburg/doma_dach...,2749486551,Дом 120 м² на участке 8 сот.,"Дома, дачи, коттеджи",Алексей,8сот,13700000,114167,1718600021000,...,NaN,NaN,NaN,17 июня 07:53,Уютный дом в садоводстве Снт «Семиозерье-2».\n...,Санкт-Петербург,60.312823,29.305870,sale,17.06.2024
22975,22975,https://www.avito.ru/sankt-peterburg/doma_dach...,2426077854,"Дом 63,2 м² на участке 6,1 сот.","Дома, дачи, коттеджи",Денис Александрович,"6,1сот",14900000,235759,1716265247000,...,NaN,NaN,NaN,21 мая 07:20,Готовое к проживанию домовладении в окружение ...,Санкт-Петербург,60.319431,29.297447,sale,21.05.2024
22976,22976,https://www.avito.ru/sankt-peterburg/doma_dach...,4068909407,"Дом 52,1 м² на участке 15 сот.","Дома, дачи, коттеджи",Елена,15сот,8000000,153551,1717144984000,...,NaN,NaN,NaN,31 мая 11:43,Продается зимний рубленный 2 эт дом.\n2012 г п...,Санкт-Петербург,60.304096,29.275332,sale,31.05.2024


In [99]:
pd.merge(dt[dt.columns[1:]], new_d, how = 'outer', on = 'url').drop_duplicates()#.isna().sum()

,url,id,name,type,seller,square,price,price_sot,timestamp_published,address,...,Транспортная доступность,Инфраструктура,Способ продажи,Расстояние от КАД,Интернет и ТВ,Электричество,Отопление,Водоснабжение,Газ,Канализация
0,https://www.avito.ru/sankt-peterburg/doma_dach...,3480989002,"Дом 126 м² на участке 4,5 сот.","Дома, дачи, коттеджи",Домакоттеджи,"4,5сот",12600000,100000,1716437193000,Санкт-Петербург,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.avito.ru/sankt-peterburg/doma_dach...,3480989002,"Дом 126 м² на участке 4,5 сот.","Дома, дачи, коттеджи",Домакоттеджи,"4,5сот",12600000,100000,1716437193000,"Санкт-Петербург, Курортный район, посёлок Бело...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.avito.ru/sankt-peterburg/doma_dach...,2489577969,Дом 150 м² на участке 6 сот.,"Дома, дачи, коттеджи",Домакоттеджи,6сот,9900000,66000,1716437259000,Санкт-Петербург,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.avito.ru/sankt-peterburg/doma_dach...,2489577969,Дом 150 м² на участке 6 сот.,"Дома, дачи, коттеджи",Домакоттеджи,6сот,9900000,66000,1716437259000,"Санкт-Петербург, Курортный район, посёлок Бело...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.avito.ru/sankt-peterburg/doma_dach...,3991410367,Дом 130 м² на участке 12 сот.,"Дома, дачи, коттеджи",Мария,12сот,33000000,253846,1716986344000,Санкт-Петербург,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23275,https://www.avito.ru/sankt-peterburg_lomonosov...,3747533991,"Дом 92 м² на участке 6,2 сот.","Дома, дачи, коттеджи",Clevergrad,"6,2сот",8500000,92391,1718431631000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23276,https://www.avito.ru/sankt-peterburg_lomonosov...,4031625879,Дом 123 м² на участке 15 сот.,"Дома, дачи, коттеджи",Евгения,15сот,19500000,158537,1718262849000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23277,https://www.avito.ru/sankt-peterburg_lomonosov...,3348226360,Дом 140 м² на участке 10 сот.,"Дома, дачи, коттеджи",Михаил,10сот,15100000,107857,1718105083000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23278,https://www.avito.ru/sankt-peterburg/doma_dach...,3926178096,Коттедж 250 м² на участке 15 сот.,"Дома, дачи, коттеджи",АН Петергоф,15сот,28000000,112000,1717365365000,"Ленинградская область, Ломоносовский район, Пе...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
dt[dt.columns[1:]].isna().sum()

url                        0
id                         0
name                       0
type                       0
seller                     0
square                     0
price                      0
price_sot                  0
timestamp_published        0
address                 1693
picture                   17
highway                 2264
time_to_metro          22978
distance_to_metro      22978
date_published             0
description                1
location                   0
latitude                   0
longitude                  0
type_deal                  0
date_supple                0
dtype: int64

In [91]:
new_d.isna().sum()

Количество комнат           2453
Площадь дома                2453
Площадь участка             2453
Этажей в доме               2453
Для отдыха                  3188
Категория земель            2453
Год постройки               2724
Материал стен               2453
Терраса или веранда         3002
Санузел                     2642
Коммуникации                3156
Ремонт                      2453
Парковка                    2845
Транспортная доступность    2775
Инфраструктура              2814
Способ продажи              2853
Расстояние от КАД           2453
url                            0
Интернет и ТВ               3105
Электричество               2956
Отопление                   3067
Водоснабжение               2974
Газ                         3081
Канализация                 3043
dtype: int64

In [89]:
new_d.to_excel('O:\Отдел стратегического развития\Данные\Avito\suburb\Недвижимость\pers_info1.xlsx', index = False)

In [6]:
s = r'O:\Отдел стратегического развития\Данные\Avito\flats\Supply.xlsx'
sl = r'O:\Отдел стратегического развития\Данные\Avito\flats\черновики\Sales — копия.xlsx'

In [7]:
d1 = pd.read_excel(sl)

BadZipFile: Bad CRC-32 for file 'xl/worksheets/sheet1.xml'

In [148]:
d1['square'] = d1['square'].str.replace('м²', '').str.replace('га', '')
# d1['typeland'] = d1['name'].apply(lambda x: '' if not re.search('\([\w]{3}', x) else re.search('\([\w]{3}', x)[0]).str.replace('(', '')

In [143]:
#d1['square_home']

def rom(x: str):
    if 'студия' not in x:
        return '' if not re.findall('([\d]{1})-к.', x) else re.findall('([\d]{1})-к', x)[0]
    return 'студия'

def floor(x: str):
    return '' if not re.findall('([\d]{1,})/', x) else re.findall('([\d]{1,})/', x)[0]

def floors(x: str):
    return '' if not re.findall(r'/([\d]{1,})', x) else re.findall(r'/([\d]+)', x)[0]

d1['rooms'] = d1['name'].apply(lambda x: rom(x))
# d1 = d1.drop(columns = ['district'] )
d1['date_supple'] = d1['timestamp_published'].apply(lambda x: datetime.fromtimestamp(int(str(x)[:-3])).strftime('%d.%m.%Y'))

d1['floor'] =  d1['name'].apply(lambda x: floor(x))
d1['floors'] = d1['name'].apply(lambda x: floors(x))


In [146]:
with pd.ExcelWriter(s, engine='xlsxwriter', engine_kwargs={'options' : { 'strings_to_urls': False} }) as writer:
        d1.to_excel(writer, 'Influence on Policy', index = False)

In [29]:
re.findall(r'([\d]{1})-к.', d1['name'][0])

['2']